# SUDOKU

but this time we solve it with a sat solver

In [4]:
from pysat.formula import *
from pysat.solvers import Solver
import numpy as np

Let's dive into the idea. What are the constraints? How do we define our domain? 
We will use a 9 x 9 x 9 boolean space representation of the grid. Every cell will be represented as a vectors. 
In that vector every value is 0 excepted the i-th value that will be 1 and represent the actual value of the cell.

Let's make the grid.

In [5]:
I = [i for i in range (3)] # rows
J = [j for j in range (3)] # columns
V = [v for v in range (3)] # values
grid = np.array([[[Atom(f'{i+1}_{j+1}_{v+1}') for v in V ]for j in J] for i in I]) 

In [41]:
def exclusivity_in_a_set(set_t):
    total_formula = []
    for var in set_t:
        formula = []
        for oth_var in set_t:
            if var != oth_var:
                formula.append(And(var, Neg(oth_var)))
    print(Or(formula for formula in total_formula))
    return Or(formula for formula in total_formula)

In [42]:
a,b,c = Atom('a'), Atom('b'), Atom('c')
formula = exclusivity_in_a_set([a, b, c])
print(formula)

AttributeError: 'generator' object has no attribute 'type'

In [8]:
with Solver(name='g3') as solver:
    solver.append_formula([c for c in formula])
    if solver.solve():
        print("SAT")
        model = solver.get_model()
        idp = Formula.export_vpool().id2obj
        for k in model:
            if k > 0:
                print(f'{id(k)} {abs(k)}')
            else:
                print(f'O')
    else:
        print("UNSAT")

c:\Users\39346\AppData\Local\Programs\Python\Python312\Lib\site-packages\pysat\formula.py:11: SyntaxWarning: invalid escape sequence '\m'
  """
c:\Users\39346\AppData\Local\Programs\Python\Python312\Lib\site-packages\pysat\formula.py:1451: SyntaxWarning: invalid escape sequence '\i'
  """
c:\Users\39346\AppData\Local\Programs\Python\Python312\Lib\site-packages\pysat\formula.py:1607: SyntaxWarning: invalid escape sequence '\e'
  """
c:\Users\39346\AppData\Local\Programs\Python\Python312\Lib\site-packages\pysat\formula.py:1673: SyntaxWarning: invalid escape sequence '\i'
  """
c:\Users\39346\AppData\Local\Programs\Python\Python312\Lib\site-packages\pysat\formula.py:1829: SyntaxWarning: invalid escape sequence '\e'
  """
c:\Users\39346\AppData\Local\Programs\Python\Python312\Lib\site-packages\pysat\formula.py:2161: SyntaxWarning: invalid escape sequence '\l'
  """
c:\Users\39346\AppData\Local\Programs\Python\Python312\Lib\site-packages\pysat\formula.py:2223: SyntaxWarning: invalid escape 

AttributeError: 'list' object has no attribute '_clausify'